In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import accuracy_score
from tensorflow.keras.layers import Dense, Input, BatchNormalization
from tensorflow.keras.models import Sequential
import keras
from keras.optimizers import Adam
from xgboost import XGBClassifier

2025-04-10 06:19:17.442645: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744258757.663900   96496 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744258757.742047   96496 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-10 06:19:18.137891: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')

In [3]:
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
train_df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


Filling NaN

In [3]:
train_df.isna().sum().sort_values()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Embarked         2
Age            177
Cabin          687
dtype: int64

In [4]:
train_df = train_df.drop(columns=['Cabin', 'Ticket', 'Name'])
test_df = test_df.drop(columns=['Cabin', 'Ticket', 'Name'])

In [41]:
train_df

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,male,22.0,1,0,7.2500,S
1,2,1,1,female,38.0,1,0,71.2833,C
2,3,1,3,female,26.0,0,0,7.9250,S
3,4,1,1,female,35.0,1,0,53.1000,S
4,5,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...,...
886,887,0,2,male,27.0,0,0,13.0000,S
887,888,1,1,female,19.0,0,0,30.0000,S
888,889,0,3,female,NaN,1,2,23.4500,S
889,890,1,1,male,26.0,0,0,30.0000,C


In [5]:
x = train_df.drop(columns=['PassengerId','Survived'])
y = train_df['Survived']

In [48]:
y.shape

(891,)

In [6]:
ohe_columns = ['Sex','Embarked']
num_columns = train_df.select_dtypes(exclude='object').columns
num_columns = num_columns.drop(['PassengerId','Survived'])

In [7]:
num_columns

Index(['Pclass', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object')

In [7]:
ohe_pipeline = Pipeline(steps=[
    ('impute',SimpleImputer(strategy='most_frequent')),
    ('ohe',OneHotEncoder(handle_unknown='ignore',sparse_output=False))
])

In [8]:
num_pipeline = Pipeline(steps=[
    ('impute',SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

In [9]:
cols_trans = ColumnTransformer(transformers=[
    ('ohe',ohe_pipeline, ohe_columns),
    ('nums', num_pipeline, num_columns)
], remainder='passthrough',n_jobs=-1)

In [10]:
pipeline = Pipeline(steps=[
    ('preprocessing', cols_trans)
])

In [11]:
x_preprocessed = pipeline.fit_transform(x)

In [12]:
x_train, x_test, y_train, y_test = train_test_split(x_preprocessed, y, test_size=0.2, random_state=23)

In [13]:
logreg = LogisticRegression()

In [14]:
logreg.fit(x_train,y_train)

LogisticRegression()

In [15]:
y_pred = logreg.predict(x_test)
accuracy_score(y_pred=y_pred, y_true=y_test)

0.7486033519553073

## NN

In [71]:
model = Sequential([
    Input((x_preprocessed.shape[1],)),
    Dense(units=32, activation='relu'),
    BatchNormalization(),
    Dense(units=16, activation='relu'),
    BatchNormalization(),
    Dense(units=1, activation='sigmoid')
])
model.compile(optimizer=Adam(learning_rate=0.001), loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy'])


In [72]:
model.fit(x_preprocessed,y,epochs=20)

Epoch 1/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 4s 41ms/step - accuracy: 0.6129 - loss: 0.6655
Epoch 2/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7704 - loss: 0.5046
Epoch 3/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8058 - loss: 0.4548
Epoch 4/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7952 - loss: 0.4531
Epoch 5/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8132 - loss: 0.4267
Epoch 6/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8215 - loss: 0.4303
Epoch 7/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8091 - loss: 0.4268
Epoch 8/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8185 - loss: 0.4130
Epoch 9/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8282 - loss: 0.3957
Epoch 10/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8145 - loss: 0.4348
Epoch 11/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8147 - loss: 0.4095
Epoch 12/20
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8434 - l

In [18]:
test_x = test_df.drop(columns=['PassengerId'])
test_preprocessed = pipeline.fit_transform(test_x)

### XGBoost

In [16]:
xgb = XGBClassifier(n_estimators=500, learning_rate=0.1, random_state=92, verobose=1, early_stopping_rounds=50)
xgb.fit(x_train,y_train, eval_set=[(x_test,y_test)])

[0]	validation_0-logloss:0.61583
[1]	validation_0-logloss:0.58508


[06:22:40] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "verobose" } are not used.



[2]	validation_0-logloss:0.56029
[3]	validation_0-logloss:0.53991
[4]	validation_0-logloss:0.52379
[5]	validation_0-logloss:0.51005
[6]	validation_0-logloss:0.49976
[7]	validation_0-logloss:0.48984
[8]	validation_0-logloss:0.48275
[9]	validation_0-logloss:0.47618
[10]	validation_0-logloss:0.46888
[11]	validation_0-logloss:0.46524
[12]	validation_0-logloss:0.46167
[13]	validation_0-logloss:0.45805
[14]	validation_0-logloss:0.45368
[15]	validation_0-logloss:0.45026
[16]	validation_0-logloss:0.44777
[17]	validation_0-logloss:0.44669
[18]	validation_0-logloss:0.44564
[19]	validation_0-logloss:0.44601
[20]	validation_0-logloss:0.44416
[21]	validation_0-logloss:0.44515
[22]	validation_0-logloss:0.44359
[23]	validation_0-logloss:0.44407
[24]	validation_0-logloss:0.44426
[25]	validation_0-logloss:0.44397
[26]	validation_0-logloss:0.44615
[27]	validation_0-logloss:0.44667
[28]	validation_0-logloss:0.44638
[29]	validation_0-logloss:0.44581
[30]	validation_0-logloss:0.44761
[31]	validation_0-logl

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=50,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=500, n_jobs=None,
              num_parallel_tree=None, random_state=92, ...)

# Exporting

In [ ]:
test_df['Survived'] = logreg.predict(test_preprocessed)

In [82]:
test_df['Survived'] = model.predict(test_preprocessed)
test_df['Survived'] = (test_df['Survived']>0.5).astype(int)

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


In [19]:
test_df['Survived'] = xgb.predict(test_preprocessed)

In [20]:
test_df[['PassengerId','Survived']].to_csv('xgboost.csv', index=False)